# Example using `python2`

This notebook demonstrates the usage of `toolkitICL` using `python2` to write the configuration file.

In [ ]:
import h5py
import numpy as np
import subprocess
from __future__ import print_function

def type_to_name(T):
    if T == np.int8:
        return "char"
    elif T == np.uint8:
        return "uchar"
    elif T == np.int16:
        return "short"
    elif T == np.uint16:
        return "ushort"
    elif T == np.int32:
        return "int"
    elif T == np.uint32:
        return "uint"
    elif T == np.int64:
        return "long"
    elif T == np.uint64:
        return "ulong"
    elif T == np.float32:
        return "float"
    elif T == np.float64:
        return "double"
    else:
        raise TypeError("Unknown type")


# Define an OpenCL kernel and write it to a file. Of course, you can
# write the kernel file in an editor for more complex use cases.
copy_kernel = """
#ifdef cl_khr_fp64
  #pragma OPENCL EXTENSION cl_khr_fp64 : enable
#else
  #error \"IEEE-754 double precision not supported by OpenCL implementation.\"
#endif

kernel void copy(global COPYTYPE const* in, global COPYTYPE* out)
{
  const int gid = get_global_id(0);
  out[gid] = in[gid];
}
"""
kernel_url = "copy_kernel_py2.cl"
with open(kernel_url, "w") as io:
    io.write(copy_kernel)

    
# input parameters
LENGTH = 32
T = np.float64

# generate input/output arrays for OpenCL
input = np.linspace(0, LENGTH-1, num=LENGTH, dtype=T)
output = np.zeros(LENGTH, dtype=T)

# write the configuration file
filename = "copy_" + type_to_name(T) + "_test_py2.h5"
with h5py.File(filename, 'w') as io:
    # compiler arguments for OpenCL
    io.create_dataset("Kernel_Settings", data="-DCOPYTYPE="+type_to_name(T))
    
    # URL of the file containing the kernels
    io.create_dataset("Kernel_URL", data=kernel_url)
    # Instead, you could also io.create_dataset("Kernel_Source", data=copy_kernel).
    
    # kernels to be executed
    io.create_dataset("Kernels", data=["copy"])

    # OpenCL ranges
    tmp_range = np.array([LENGTH, 1, 1], dtype=np.int32)
    io.create_dataset("Global_Range", data=tmp_range)

    tmp_range[:] = 0
    io.create_dataset("Local_Range", data=tmp_range)
    io.create_dataset("Range_Start", data=tmp_range)

     # data
    io.create_dataset("Data/in", data=input)
    io.create_dataset("Data/out", data=output)

    
# call toolkitICL
p = subprocess.Popen(["toolkitICL", "-c", filename], stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
while p.poll() is None:
    l = p.stdout.readline()
    print(l, end="")
print(p.stdout.read())


# check results
out_filename = "out_" + filename
with h5py.File(out_filename, 'r') as io:
    in_test = np.array(io["Data/in"])
if not np.all(in_test.flatten() == input):
    print("Wrong result for 'in'")

with h5py.File(out_filename, 'r') as io:
    out_test = np.array(io["Data/out"])
if not np.all(out_test.flatten() == input):
    print(out_test.flatten())
    print(input)
    print("Wrong result for 'out'")

print("Finished successfully.")

In [ ]:
import os

# remove temporary files
def rm_if_exists(s):
    if os.path.isfile(s):
        os.remove(s)

rm_if_exists(filename)
rm_if_exists(kernel_url)
rm_if_exists(out_filename)
os.listdir("./")